# Siamese network, 02062017

In [7]:
from datetime import datetime
from IPython.display import SVG

import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding, Bidirectional, Lambda
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, ProgbarLogger, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback

from utils import load_embeddings, extract_questions_from_dataframe, save_submission

%load_ext autoreload
%autoreload 2
pd.set_option('max_colwidth', 250)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
class Config(object):
    VOCABULARY_SIZE = 1193514
    EMBEDDING_DIMENSION = 200
    OFFSET = 3
    OOV_TOKEN = 0  # out of vocabulary
    EOS_TOKEN = 1  # end of sentence
    PAD_TOKEN = 2  # padding to max sentence length
    MAX_SENTENCE_LENGTH = 60
    DENSE_LAYER_SIZE = 150
    DROPOUT = 0.4
    
    def stamp(self, comment):
        return '{date:%Y%m%d_%H%M}_{comment}'.format(
            date=datetime.now(), comment=comment)

In [4]:
%%time
train_dataframe = pd.read_csv('train.csv')
current_config = Config()

embedding_weights, word2idx = load_embeddings(
    'glove.twitter.27B.200d.txt',
    config=current_config
)

questions_A, questions_B, labels = extract_questions_from_dataframe(
    train_dataframe, 
    config=current_config,
    word2idx=word2idx,
    prediction_mode=False
)

404290 preprocessed questions loaded from disk


CPU times: user 1min 56s, sys: 5.4 s, total: 2min 1s
Wall time: 2min 13s


In [4]:
questions_A.shape

(404290, 60)

In [13]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
(1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def euclidean_distance(vectors):
    x, y = vectors
    return K.sqrt(
        K.maximum(
            K.sum(K.square(x - y), axis=1, keepdims=True), 
            K.epsilon()
        )
    )

shared_lstm_layer = Bidirectional(
    LSTM(units=100, 
         return_sequences=False, 
         dropout=0.3)
)

shared_embedding_layer = Embedding(
    input_dim=current_config.VOCABULARY_SIZE + current_config.OFFSET, 
    output_dim=current_config.EMBEDDING_DIMENSION, 
    input_length=current_config.MAX_SENTENCE_LENGTH,
    weights=[embedding_weights],
    trainable=False
)

input_A = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_A = shared_embedding_layer(input_A)
sentence_representation_A = shared_lstm_layer(embeddings_A)

input_B = Input(shape=(current_config.MAX_SENTENCE_LENGTH,))
embeddings_B = shared_embedding_layer(input_B)
sentence_representation_B = shared_lstm_layer(embeddings_B)


distance = Lambda(euclidean_distance)(
    [sentence_representation_A, 
    sentence_representation_B]
)

predictions = Dense(1, activation='sigmoid')(distance)

model = Model(inputs=[input_A, input_B], outputs=predictions)
model.compile(optimizer='rmsprop',
              loss=contrastive_loss,
              metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
input_8 (InputLayer)             (None, 60)            0                                            
____________________________________________________________________________________________________
embedding_4 (Embedding)          (None, 60, 200)       238703400   input_7[0][0]                    
                                                                   input_8[0][0]                    
____________________________________________________________________________________________________
bidirectional_4 (Bidirectional)  (None, 200)           240800      embedding_4[0][0]       

In [14]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = ModelCheckpoint(
    monitor='val_loss',
    filepath=current_config.stamp(comment='1') + '.h5', 
    save_best_only=True, 
    save_weights_only=True
)
tensorboard = TensorBoard()
tqdm_bar = TQDMNotebookCallback()

In [15]:
%%time

training_logs = model.fit(
    x=[questions_A, questions_B], 
    y=labels, 
    epochs=200, 
    batch_size=1024,
    validation_split=0.2, 
    callbacks=[early_stopping, model_checkpoint, tensorboard, tqdm_bar]
)

Train on 800 samples, validate on 200 samples


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Epoch 5/5


800/800 [==============================] - 10s - loss: 0.2900 - acc: 0.6250 - val_loss: 0.2722 - val_acc: 0.5800



CPU times: user 2min 33s, sys: 41.7 s, total: 3min 15s
Wall time: 1min 48s


In [38]:
!ls

20170430_2248_1.h5            extracted_questions_train.npz
20170501_0126_1.csv           glove.twitter.27B.100d.txt
20170501_0326_1.h5            glove.twitter.27B.200d.txt
20170501_1742_1.csv           glove.twitter.27B.25d.txt
20170504_2334_1.h5            glove.twitter.27B.50d.txt
20170505_0156_1.csv           logs
20170505_0211_1.h5            logs_2
20170505_0304_1.h5            logs_3
20170505_1840_1.csv           sample_submission.csv
20170507_2315_1.csv           script.py
20170509_0055_1.h5            tensorflow
20170509_2249_1.h5            test.csv
20170509_2252_1.h5            train.csv
20170509_2253_1.h5            utils.py
29042017_1.csv                vanilla_lstms_1.ipynb
README.md                     vanilla_lstms_2.ipynb
__pycache__                   wiki.en.vec


In [20]:
questions_B.shape

(404290, 60)

In [39]:
model.load_weights('20170509_2253_1.h5')
min(training_logs.history['val_loss'])

0.49221511489207603

In [40]:
test_dataframe = pd.read_csv('test.csv')
test_questions_A, test_questions_B, _ = extract_questions_from_dataframe(
    test_dataframe,
    config=current_config,
    word2idx=word2idx,
    prediction_mode=True
)

No saved file, preprocessing from scratch
2345796 questions preprocessed


In [41]:
%%time
predictions = model.predict(
    x=[test_questions_A, test_questions_B], 
    batch_size=8192, 
    verbose=1
)

2345796/2345796 [==============================] - 6118s  
CPU times: user 4h 24min 57s, sys: 1h 7min 12s, total: 5h 32min 9s
Wall time: 1h 41min 58s


In [42]:
save_submission(predictions, current_config)

,is_duplicate
0,0.236641
1,0.299976
2,0.194677
3,0.176530
4,0.413835
5,0.319847
6,0.656540
7,0.741822
8,0.487091
9,0.129493


In [ ]:
!wc -l 20170505_0156_1.csv

In [ ]:
predictions[:100]

In [ ]:
test_dataframe